# Metadata creation

In [ ]:
import os
import sys
import json
import pandas as pd

sys.path.insert(0, os.path.abspath('../../src/scraping'))
import youtube_requesting as ytr
import metadata as mdata

ROOT_DIR = "../../"

In [ ]:
with open(ROOT_DIR + "config/" + "config-scraping.json") as f:
    cfg = json.load(f)
with open(ROOT_DIR + "api_key.json") as json_file:
    cred = json.load(json_file)

In [ ]:
data = mdata.fetch_raw_data(ROOT_DIR, cfg)

api_key = cred['api_key']

df = mdata.make_mdata_df(data["04_16_20"], api_key)

In [ ]:
df

In [ ]:
game = cfg["selected-game"]
df.to_csv(ROOT_DIR + cfg["videos-dir"] + game + '/{}_metadata.csv'.format(game), index=False)

# Downloading thumbnails

In [ ]:
mdata.download_df_thumbs(df, ROOT_DIR + cfg["thumbnails-dir"] + cfg["selected-game"] + '/', res="default")

In [ ]:
df = pd.read_csv(ROOT_DIR + cfg["videos-dir"] + cfg["selected-game"] + "/fortnite_metadata.csv")

# Statistics

### Desired functions:
- Given a video_id, get the surrounding videos and z-score that video in comparison.
- Get the basic video statistics surrounding a video
    - Likes
    - Dislikes
    - Views
    - Subscriber count
    - Video length(?)
- Combination of all stats to form a "success score"

### Note on api costs
Calculate those costs here:
https://developers.google.com/youtube/v3/determine_quota_cost

In [ ]:
# Parameters

# When calculating z-score of a video, grab the +- x videos surrounding the target video.
nearest_videos = 5

## Possible success metrics
Available measurements:
- date
- views (COUNT)
- likes (COUNT)
- dislikes (COUNT)
- comments (COUNT)

Metadata recommendations from here (page 6-7):
https://www.sciencedirect.com/science/article/pii/S187705091731757X/pdf?md5=5961456d3adc0f65e03cff17279317d2&pid=1-s2.0-S187705091731757X-main.pdf

Get a number of videos from the parent channel closest to the selected video in date and zscore the available measurements, then average those measurements.

Weight views higher?

Make another function that can take in an input of a list of video ids

## Statistics
#### (WIP)
Add in a measurement for video time length

Expecting a new input

`
{'video_id': '4uf2x-O_pBw',
 'position': '0',
 'channel_videos': ['4uf2x-O_pBw', '8cu4qAoeXow', 'TZK-sk5Yvfw'],
 'channel_id':str}
`

In [ ]:
def get_channel_vids_stats(videos):
    stats = []
    for v in videos:
        curr_vid = ytr.request_video_details(v,
                                             api_key,
                                             api_service_name,
                                             api_version)['items'][0]
        stats.append(ytr.get_vid_stats(curr_vid))
        time.sleep(1)
    
    return pd.DataFrame(stats)

In [ ]:
def video_success(in_dict):
    vid_stats = get_channel_vids_stats(in_dict["channel_videos"])
    
    # A lower number position means the video is more popular.
    position_weight = 1.5
    position_score = (1 - (in_dict["position"] / len(in_dict["channel_videos"]))) * position_weight
    
    zscores = vid_stats[["comments", "dislikes", "likes", "views"]].apply(scipy.stats.zscore)
    
    return position_score + zscores.loc[in_dict["position"]].sum()

In [ ]:
def video_success(row, weights=[1 for _ in len(row)]):
    position_score = (1 - (row.loc["position"] / len(in_dict["channel_videos"]))) * position_weight

In [ ]:
df.keys()

In [ ]:
row = df.iloc[0]

In [ ]:
row